In [ ]:
# Dependencies
! pip install census
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
#import gmaps

# Census API Keys
from config import (api_key)
from config import gkey
c = Census(api_key, year=2018)

In [ ]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E", "B23025_005E", "B03001_003E","B02001_003E",
                          "B02001_005E","B06009_005E","B06009_006E"), {'for': 'zip code tabulation area:*'})


# Convert to DataFrame

census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "B03001_003E": "Number of Hispanic Residents",
                                      "B02001_003E": "Number of African American Residents",
                                      "B02001_005E": "Number of Asian Residents",
                                      "B06009_005E": "Number of Residents w/Bachelor's Degree",
                                      "B06009_006E": "Number of Residents w/Graduate Degree",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = census_df["Poverty Count"]/census_df["Population"]*100

# Add in Employment Rate (Employment Count / Population)
census_df["Unemployment Rate"] = census_df["Unemployment Count"]/census_df["Population"]*100


# Final DataFrame
census_df = census_df.loc[(census_df['Zipcode'] == "20007")|(census_df['Zipcode'] == "20016")|
                         (census_df['Zipcode'] == "20008")| (census_df['Zipcode'] == "20009")| (census_df['Zipcode'] == "20010")
                         |(census_df['Zipcode'] == "20011")| (census_df['Zipcode'] == "20012")| (census_df['Zipcode'] == "20013")
                         |(census_df['Zipcode'] == "20390")| (census_df['Zipcode'] == "20015")| (census_df['Zipcode'] == "20373")
                         | (census_df['Zipcode'] == "20319")| (census_df['Zipcode'] == "20317")| (census_df['Zipcode'] == "20064")
                         | (census_df['Zipcode'] == "20057")| (census_df['Zipcode'] == "20052")| (census_df['Zipcode'] == "20037")
                         | (census_df['Zipcode'] == "20036")| (census_df['Zipcode'] == "20032")| (census_df['Zipcode'] == "20024")
                         | (census_df['Zipcode'] == "20020")| (census_df['Zipcode'] == "20019")| (census_df['Zipcode'] == "20018")
                         | (census_df['Zipcode'] == "20017")| (census_df['Zipcode'] == "20001")| (census_df['Zipcode'] == "20002")
                         | (census_df['Zipcode'] == "20003")| (census_df['Zipcode'] == "20004")| (census_df['Zipcode'] == "20005")
                         | (census_df['Zipcode'] == "20006")]
census_df
census_df['Coords']= ''
census_df

In [ ]:
census_df["Percent Minority Population"] = (census_df["Number of Asian Residents"]+
                                            census_df["Number of Hispanic Residents"]+
                                            census_df["Number of African American Residents"])/census_df["Population"]*100
census_df["Percent of Residents w/Bachelor's Degree or Higher"] = (census_df["Number of Residents w/Bachelor's Degree"]+
                                                                   census_df["Number of Residents w/Graduate Degree"])/census_df["Population"]*100
census_final = census_df[["Zipcode", "Population","Per Capita Income", "Poverty Rate", "Percent of Residents w/Bachelor's Degree or Higher", "Percent Minority Population"]]
census_final["Zipcode"] = census_final["Zipcode"].astype(int)
census_final

In [ ]:
# Finding lat/lang so we can get a nearby address.

geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json'


for index,row in census_df.iterrows():
    location = row['Zipcode']
    params = {
    'address': location,
    'key': gkey
    }
    response = requests.get(geocode_url, params=params)
    response = response.json()

    lat = response["results"][0]["geometry"]["location"]["lat"]
    lng = response["results"][0]["geometry"]["location"]["lng"]
    
    origin_latlng = f"{lat},{lng}"
    census_df.loc[index, 'Coords'] = origin_latlng

census_df.head()

In [ ]:
census_df.sort_values("Household Income")